In [2]:
using HiGHS
using SDDP
using Statistics

In [3]:
graph=SDDP.UnicyclicGraph(0.95; num_nodes = 3)

Root
 0
Nodes
 1
 2
 3
Arcs
 0 => 1 w.p. 1.0
 1 => 2 w.p. 1.0
 2 => 3 w.p. 1.0
 3 => 1 w.p. 0.95


In [4]:
model = SDDP.PolicyGraph(
    graph,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do sp, t
    @variable(sp, 5 <= x <= 15, SDDP.State, initial_value = 10)
    @variable(sp, g_t >= 0)
    @variable(sp, g_h >= 0)
    @variable(sp, s >= 0)
    @constraint(sp, balance, x.out - x.in + g_h + s == 0)
    @constraint(sp, demand, g_h + g_t == 0)
    @stageobjective(sp, s + t * g_t)
    SDDP.parameterize(sp, [[0, 7.5], [3, 5], [10, 2.5]]) do w
        set_normalized_rhs(balance, w[1])
        return set_normalized_rhs(demand, w[2])
    end
end

A policy graph with 3 nodes.
 Node indices: 1, 2, 3


In [5]:
SDDP.train(model, iteration_limit = 100)

------------------------------------------------------------------------------
          SDDP.jl (c) Oscar Dowson and SDDP.jl contributors, 2017-23

Problem
  Nodes           : 3
  State variables : 1
  Scenarios       : Inf
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (6, 6)
    VariableRef in MOI.LessThan{Float64}    : (1, 1)
    VariableRef in MOI.GreaterThan{Float64} : (5, 5)
    AffExpr in MOI.EqualTo{Float64}         : (2, 2)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [1e+00, 1e+00]
  Non-zero Objective range  [1e+00, 3e+00]
  Non-zero Bounds range     [5e+00, 2e+01]
  Non-zero RHS range        [2e+00, 1e+01]
No problems detected

 Iteration    Simulation       Bound         Time (s)    Proc. ID   # Solves
        1    4.350000e+01   4.437148e+00   5.719000e+00   

In [6]:
sims = SDDP.simulate(model, 100, [:g_t])
mu = round(mean([s[1][:g_t] for s in sims]), digits = 2)
println("On average, $(mu) units of thermal are used in the first stage.")

On average, 2.18 units of thermal are used in the first stage.


In [7]:
sims = SDDP.simulate(model, 100, [:g_h])
mu = round(mean([s[1][:g_h] for s in sims]), digits = 2)
println("On average, $(mu) units of hydro are used in the first stage.")

On average, 2.8 units of hydro are used in the first stage.


In [8]:
V = SDDP.ValueFunction(model[1])
cost, price = SDDP.evaluate(V, x = 10)

(233.5281478126708, Dict(:x => -0.6591125751212569))